In [45]:
import pandas as pd
import re


In [89]:
df = pd.read_csv('../data/CYSOanonymized.csv')
df.columns

Index(['Unnamed: 0', 'uni', 'instrument', 'gradyear', 'major', 'finaid',
       'race', 'gender', 'm_employ', 'm_job', 'd_employ', 'homezip', 'school',
       's_address', 's_state', 's_zip', 's_ensemble', 'pt', 'studentID'],
      dtype='object')

In [221]:
# read in data
df = pd.read_csv('../data/CYSOanonymized.csv')

# set financial aid status to 0 if NaN

df['finaid'].fillna(0,inplace=True)

# parse race into different rows
df['race'].fillna('Other',inplace=True)
df['caucasian']=0
df['african_american']=0
df['native_american']=0
df['latino']=0
df['asian']=0
df['other_race']=0
df['race_num']=0

for index, row in df.iterrows():
    if re.search('cauc', str(row['race']), re.IGNORECASE):
        df.loc[index,'caucasian']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('african', str(row['race']), re.IGNORECASE):
        df.loc[index,'african_american']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('native', str(row['race']), re.IGNORECASE):
        df.loc[index,'native_american']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('latino', str(row['race']), re.IGNORECASE):
        df.loc[index,'latino']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('asian', str(row['race']), re.IGNORECASE):
        if not re.search('caucasian', str(row['race']), re.IGNORECASE):
            df.loc[index,'asian']=1
            df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search('other', str(row['race']), re.IGNORECASE):
        df.loc[index,'other_race']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1
    if re.search("`", str(row['race']), re.IGNORECASE):
        df.loc[index,'other_race']=1
        df.loc[index,'race_num']=df.loc[index,'race_num']+1

# compile gender
df['male']=0
df['female']=0

df.gender = df.gender.str.strip()
df.gender.replace('Male','M',inplace=True)
df.gender.replace('Female','F',inplace=True)

for index, row in df.iterrows():
    if row['gender']=='M':
        df.loc[index, 'male']=1
    if row['gender']=='F':
        df.loc[index,'female']=1

# compile major

df['music']=0

music_terms = ['performance','music','violin','songwriting','perfomance','bass','cello','viola','jazz']

for index, row in df.iterrows():
    for music_term in music_terms:
        if re.search(music_term,str(row['major']),re.IGNORECASE):
            df.loc[index,'music']=1
    

# map in incomes

income = pd.read_csv('../data/income_zip.csv')
df['homezip']=pd.to_numeric(df['homezip'], errors='coerce')
df = pd.merge(df,income,how='left',left_on='homezip',right_on='Zip')


# drop spurious columns and rename others
df.rename(index=str, columns={"Mean": "mean_income", "Median": "median_income"}, inplace=True)
df['mean_income']=pd.to_numeric(df['mean_income'], errors='coerce')
df['median_income'] = df['median_income'].str.replace(',','')
df['median_income']=pd.to_numeric(df['median_income'], errors='coerce')
df.drop(['Zip', 'Pop', 'Unnamed: 0','instrument','race','gender','s_ensemble',\
        'm_employ','m_job','d_employ','school','s_address','uni','pt',\
        's_state','major','homezip','s_zip','studentID','mean_income','gradyear'], axis=1, inplace=True)



# save new data structure
df.to_csv('../data/fitting_data.csv')